### 1.test M_mask calculation

In [1]:
import tensorflow as tf

In [25]:
q_len, d_len = 5, 10
q_lens = tf.constant([3, 2, 1, 3, 4], dtype=tf.int32)
d_lens = tf.constant([7, 8, 9, 6, 6], dtype=tf.int32)
q_mask = tf.sequence_mask(q_lens, q_len, dtype=tf.float32)
d_mask = tf.sequence_mask(d_lens, d_len, dtype=tf.float32)

print("q_mask shape:{}".format(q_mask.get_shape()))
print("d_mask shape:{}".format(d_mask.get_shape()))
M_mask = tf.einsum("bi,bj->bij", d_mask, q_mask)
M_mask.get_shape()

q_mask shape:(5, 5)
d_mask shape:(5, 10)


TensorShape([Dimension(5), Dimension(10), Dimension(5)])

In [26]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(q_mask))
    print("-" * 50)
    print(sess.run(d_mask))
    print("-" * 50)
    print(sess.run(M_mask))

[[ 1.  1.  1.  0.  0.]
 [ 1.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.]
 [ 1.  1.  1.  1.  0.]]
--------------------------------------------------
[[ 1.  1.  1.  1.  1.  1.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
 [ 1.  1.  1.  1.  1.  1.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  0.  0.  0.  0.]]
--------------------------------------------------
[[[ 1.  1.  1.  0.  0.]
  [ 1.  1.  1.  0.  0.]
  [ 1.  1.  1.  0.  0.]
  [ 1.  1.  1.  0.  0.]
  [ 1.  1.  1.  0.  0.]
  [ 1.  1.  1.  0.  0.]
  [ 1.  1.  1.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 1.  1.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.]
  [ 1.  0.  0.  0.  0.]
  [ 1.  0.

### 2.test attention sum

In [1]:
import tensorflow as tf
import numpy as np

In [7]:
batch_size = 3
d_len = 10
vocab_size = 20
A_len = 4
true_s_bd = np.array([0.1]*d_len*batch_size).reshape(batch_size,d_len)
true_documents_bt = np.random.randint(0,vocab_size,size=(batch_size,d_len))
true_candidates_bi = np.random.randint(0,vocab_size,size=(batch_size,A_len))

print(true_s_bd)
print(true_s_bd.shape)
print("-"*50)
print(true_documents_bt)
print(true_documents_bt.shape)
print("-"*50)
print(true_candidates_bi)
print(true_candidates_bi.shape)

[[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]]
(3, 10)
--------------------------------------------------
[[ 4  3 18 19  7  9 13  1 10 11]
 [ 5 16 11  6  4  0 16  1 11  8]
 [ 3  5  2 12  2  8 14  1 15 11]]
(3, 10)
--------------------------------------------------
[[ 3 11 19 11]
 [ 9  1 14 11]
 [13 10 10 13]]
(3, 4)


In [8]:
s_bd = tf.placeholder(dtype=tf.float32, shape=(None, d_len), name="s_bd")
documents_bt = tf.placeholder(dtype=tf.int32, shape=(None, d_len), name="documents_bt")
candidates_bi = tf.placeholder(dtype=tf.int32, shape=(None, A_len), name="candidates_bi")
y_hat_bi = tf.scan(fn=lambda prev, cur:
tf.gather(tf.unsorted_segment_sum(cur[0], cur[1], vocab_size), cur[2]),
                   elems=[s_bd, documents_bt, candidates_bi],
                   initializer=tf.Variable([0.] * A_len,dtype=tf.float32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    data = {
        s_bd:true_s_bd,
        documents_bt:true_documents_bt,
        candidates_bi:true_candidates_bi
    }
    print(sess.run(y_hat_bi,feed_dict=data))

[[ 0.1  0.1  0.1  0.1]
 [ 0.   0.1  0.   0.2]
 [ 0.   0.   0.   0. ]]
